This code works well for coarser rasters then the 250m-resolution-rasters from soilgrids. Therefore, the resampling of the clay-raster has been done in ArcGIS.

In [ ]:
import os
import mypackages.myrasters as mr
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
data_dir = os.path.join('..', 'data')
out_dir = os.path.join('..', 'output/soilgrids_prepared')

In [ ]:
# test data
"""
array = np.array([[1, 2, 255, 4, 2, 3],
                  [5, 6, 7, 8, 4, 2],
                  [255, 255, 255, 255, 4, 1],
                  [8, 7, 255, 255, 8, 1]])
#"""

In [ ]:
# test array
"""
soilm = mr.MyRaster(os.path.join(data_dir, 'soilm_test.tif'))
array = soilm.get_array()
#"""

In [ ]:
#plt.imshow(mr.mask_array(array, 255))
#plt.imshow(mr.mask_array(array, soilm.NoDataValue))

In [ ]:
%%time

#new_array = mr.resample_array_to_lower_resolution(array, 0.5, 255)
#new_array = mr.resample_array_to_lower_resolution(array, 0.25, soilm.NoDataValue)

In [ ]:
#plt.imshow(mr.mask_array(new_array, 255))
#plt.imshow(mr.mask_array(new_array, soilm.NoDataValue))

Code for resampling the clay-raster (works but needs a lot of computational power and time):

In [ ]:
clay_dir = os.path.join('..', 'output/soilgrids_prepared')

In [ ]:
clay_raster = mr.MyRaster(os.path.join(clay_dir, 'CLYPPT_ts.tif'))

In [ ]:
%%time

clay = clay_raster.get_array()

In [ ]:
%%time

new_array = mr.resample_array_to_lower_resolution(clay, 0.25, clay_raster.NoDataValue)

In [ ]:
%%time

mr.array2geotiff(new_array, os.path.join(data_dir, 'CLYPPT_ts_pyresample'),
                 clay_raster.NoDataValue, new_array.shape[1], new_array.shape[0],
                 clay_raster.xOrigin, clay_raster.yOrigin,
                 clay_raster.pixelWidth/scaling_factor, clay_raster.pixelHeight/scaling_factor)

Doing it tile for tile and put it back together (incomplete code):

In [ ]:
%%time

clay = clay_raster.get_bounded_array(-180, -179, clay_raster.yMin, clay_raster.yMax)

In [ ]:
scaling_factor = 0.25

In [ ]:
new_shape = (int(array.shape[0] * scaling_factor), int(array.shape[1] * scaling_factor))
array.shape, new_shape

In [ ]:
def get_tiles(array, scaling_factor):
    
    """
    This function calculates the x and y offsets and the x and y size of all tiles
    the input raster is cut into.
    """
    
    # number of tiles to cut old array into = shape of new array
    x_tiles = int(array.shape[1] * scaling_factor)
    y_tiles = int(array.shape[0] * scaling_factor)
    new_shape = (y_tiles, x_tiles)
    
    # tile size in old array
    x_size = array.shape[1] / x_tiles
    y_size = array.shape[0] / y_tiles

    # offset for each tile in old array
    x_offs = [0]
    y_offs = [0]
    for i in range(1, x_tiles):
        x_offs.append(int(i*x_size))
    for i in range(1, y_tiles):
        y_offs.append(int(i*y_size))

    return [x_offs, y_offs, x_size, y_size]


In [ ]:
x_tiles = new_shape[1]
y_tiles = new_shape[0]

x_offs, y_offs, x_size, y_size = get_tiles(array, scaling_factor)
#x_offs, y_offs, x_size, y_size

In [ ]:
NoDataValue = soilm.NoDataValue

no_data = np.full(new_shape, NoDataValue)

In [ ]:
new_array = np.full(new_shape, NoDataValue)

for j in range(0, y_tiles):
    for i in range(0, x_tiles):

        tile = array[y_offs[j]:(y_offs[j] + y_size), x_offs[i]:(x_offs[i] + x_size)]
        if np.array_equal(tile, no_data):
            value = NoDataValue
        else:
            mask = tile == NoDataValue
            tile_ma = np.ma.masked_array(tile, mask)
            value = tile_ma.mean()

        new_array[j][i] = value


In [ ]:
new_array[np.isnan(new_array) == True] = NoDataValue
new_array = new_array.round(0).astype('int16')
new_array

In [ ]:
plt.imshow(new_array);

In [ ]:
new_array.shape

In [ ]:
mr.array2geotiff(new_array, os.path.join(data_dir, 'soilm_yearly_mean_cut_resampled'),
                 soilm.NoDataValue, new_array.shape[1], new_array.shape[0],
                 soilm.xOrigin, soilm.yOrigin,
                 soilm.pixelWidth/scaling_factor, soilm.pixelHeight/scaling_factor)

In [ ]:
soilm.xOrigin

In [ ]:
soilm.get_array().shape

In [ ]:
array2geotiff(a, os.path.join('output/temp', 'x' + str(i+1) +'_y' + str(j+1)), -9999,
              tiles[2] * scaling_factor, tiles[3] * scaling_factor,
              bbox_xMinDeg, bbox_yMaxDeg,
              soilm.pixelWidth/scaling_factor, soilm.pixelHeight/scaling_factor)

In [ ]:
[print(old,new) for old,new in zip(array.shape,newshape)]

In [ ]:
slices = [slice(0,old, float(old)/new) for old,new in zip(array.shape,newshape)]

In [ ]:
slices

In [ ]:
coordinates = mgrid[slices]

In [ ]:
coordinates